In [1]:
using HTTP
using WGLMakie
using GLMakie
using Graphs
using JSServe
using NetworkLayout
using GraphMakie
using CSV
using DataFrames
using LinearAlgebra

### Set up API Environment

In [10]:
using HTTP

# Post login credentials to the session
login_url = "https://websvc.biocyc.org/credentials/login/"
login_data = Dict("email" => "aniketh@stanford.edu", "password" => "dcy*BWEfAm")
response = HTTP.post(login_url, login_data)

LoadError: HTTP.Exceptions.StatusError(401, "POST", "/credentials/login/", HTTP.Messages.Response:
"""
HTTP/1.1 401 Unauthorized
Date: Wed, 19 Jul 2023 23:23:57 GMT
Connection: Close
Server: AllegroServe/1.3.85
Content-Type: JS
Transfer-Encoding: chunked
Content-Encoding: GZIP


⋮
55-byte body
""")

In [3]:
flows = DataFrame(CSV.File("whole_cell_flow_data.csv"))

Row,Column1,data
,String,Float64
1,1.5.1.20-RXN-5-METHYL-THF/NAD//METHYLENE-THF/NADH/PROTON.44. (reverse),3851.0
2,1.5.1.20-RXN-CPD-1302/NAD//CPD-12996/NADH/PROTON.36. (reverse),16095.0
3,1PFRUCTPHOSN-RXN,1.23812e6
4,1TRANSKETO-RXN (reverse),7974.0
5,2-DEHYDROPANTOATE-REDUCT-RXN (reverse),1131.0
6,2-ISOPROPYLMALATESYN-RXN,56297.0
7,2-OCTAPRENYL-6-METHOXYPHENOL-HYDROX-RXN,300.0
8,2-OCTAPRENYL-6-OHPHENOL-METHY-RXN,300.0
9,2-OCTAPRENYL-METHOXY-BENZOQ-METH-RXN,300.0


In [4]:
stoichiometry = DataFrame(CSV.File("whole_cell_s_matrix (1).csv"))
metabolite_names = stoichiometry[:, "Column1"]
reaction_names = names(stoichiometry)[2:end]

n_mets = length(metabolite_names)
n_rxns = length(reaction_names)

S = Matrix(stoichiometry[:, 2:end])

489×470 Matrix{Int64}:
  1   1  0  0   0   0   0  0  0   0  0  …  0   0  0   0   0  0  0   0  0   0
 -1  -1  0  0   0   0   0  0  0   0  0     0   0  0   0   0  0  0   0  0   0
 -1  -1  1  0  -1   1  -1  1  1  -1  1     0  -1  0   1   0  0  0   1  0   1
  0   0  0  0   0   0   0  0  0   0  0     0   0  0   0   0  0  0   0  0   0
  0   0  0  0   1   0   1  0  0   1  0     0   0  0   0   0  0  0   0  0   0
  0   0  0  0  -1   0  -1  0  0  -1  0  …  0   0  0   0   0  0  0   0  0   0
  0   0  0  0   0   0   0  0  0   0  0     0   0  0   0   0  0  0   0  0   0
  0   0  0  0   0   0   0  0  0   0  0     0   1  0   0   0  0  0   0  0   0
  0   0  0  0   0   0   0  0  0   0  0     0   0  0   0   0  0  0   0  0   0
  0   0  0  0   0   0   0  0  0   0  0     0   0  0   0   0  0  0   0  0   0
  0   0  0  0   0   0   0  0  0   0  0  …  0   0  0   0   0  0  0   0  0   0
  0   0  0  0   0  -1   1  0  0   1  0     0   0  0  -1  -1  0  0  -1  0  -1
  0   0  0  0   0   0  -1  0  0  -1  0     0   0  0  

In [5]:
metabolite_names

489-element Vector{String}:
 "NAD[c]"
 "NADH[c]"
 "PROTON[c]"
 "GLUCONATE[c]"
 "NADP[c]"
 "NADPH[c]"
 "MAL[c]"
 "CARBON-DIOXIDE[c]"
 "PYRUVATE[c]"
 "Red-Thioredoxin[c]"
 "Ox-Thioredoxin[c]"
 "WATER[c]"
 "OXYGEN-MOLECULE[c]"
 ⋮
 "Enoylpimeloyl-ACP-methyl-esters[c]"
 "Pimeloyl-ACP-methyl-esters[c]"
 "3-oxo-hexanoyl-ACPs[c]"
 "R-3-hydroxyhexanoyl-ACPs[c]"
 "3-oxo-dodecanoyl-ACPs[c]"
 "R-3-hydroxydodecanoyl-ACPs[c]"
 "3-oxo-myristoyl-ACPs[c]"
 "R-3-hydroxymyristoyl-ACPs[c]"
 "Adenylated-ThiS-Proteins[c]"
 "ThiS-CoASH-proteins[c]"
 "Thi-S[c]"
 "FRU[p]"

In [6]:
reaction_names

470-element Vector{String}:
 "1.5.1.20-RXN-5-METHYL-THF/NAD//METHYLENE-THF/NADH/PROTON.44. (reverse)"
 "1.5.1.20-RXN-CPD-1302/NAD//CPD-12996/NADH/PROTON.36. (reverse)"
 "1PFRUCTPHOSN-RXN"
 "1TRANSKETO-RXN (reverse)"
 "2-DEHYDROPANTOATE-REDUCT-RXN (reverse)"
 "2-ISOPROPYLMALATESYN-RXN"
 "2-OCTAPRENYL-6-METHOXYPHENOL-HYDROX-RXN"
 "2-OCTAPRENYL-6-OHPHENOL-METHY-RXN"
 "2-OCTAPRENYL-METHOXY-BENZOQ-METH-RXN"
 "2-OCTAPRENYLPHENOL-HYDROX-RXN"
 "2.3.1.157-RXN"
 "2.5.1.19-RXN"
 "2.5.1.64-RXN"
 ⋮
 "GLUTDECARBOX-RXN__GLUTDECARBOXB-CPLX"
 "ISOCHORSYN-RXN__ENTC-MONOMER"
 "ISOCHORSYN-RXN__MENF-CPLX"
 "RXN-9535__FABB-CPLX"
 "RXN0-313__CPLX0-201 (reverse)"
 "RXN0-384__H2NEOPTERINP3PYROPHOSPHOHYDRO-MONOMER"
 "RXN0-5186__G7408-MONOMER"
 "SERINE-O-ACETTRAN-RXN__CPLX0-237"
 "SERINE-O-ACETTRAN-RXN__CYSSYNMULTI-CPLX"
 "UNDECAPRENYL-DIPHOSPHATASE-RXN[" ⋯ 57 bytes ⋯ "PROTON.64.__PGPPHOSPHAB-MONOMER"
 "URPHOS-RXN__URPHOS-CPLX"
 "maintenance_reaction"

### Processing Metabolites

In [7]:
metabolites_processed = Dict()
for meta in metabolite_names
    new = split(meta, '[', limit=2)[1]
    metabolites_processed[meta] = new
end

In [8]:
reactions_processed = Dict()
for rxn in reaction_names
    new = rxn
    if occursin("(reverse)", new)
        idx = findfirst("(reverse)", rxn)[1]
        new = rxn[1:idx - 2]
    end
    
    if occursin("__", new)
        idx = findfirst("__", new)[1]
        new = new[1:idx - 1]
    end
    
    if occursin("-RXN", new)
        idx = findfirst("-RXN", new)[4]
        new = new[1:idx]
    end
    
    reactions_processed[rxn] = new
end

In [ ]:
for meta in metabolite_names
    object_id = metabolites_processed[meta]
    
    
    
